# Dynamic few-shot prompting

What we do in this notebook:

1. Load synthetic data and embedding (generate with `model_name`).
2. Define a function for dynamic few-shot prompting (i.e., dynamically select few-shot examples based on input similarity).
3. Generate a response using `gp3-3.5-turbo` model.
4. Compare the responses with and without dynamic few-shot prompting.

In [1]:
import sqlite3
import json
from pathlib import Path
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import environ
from openai import OpenAI
import time

from utils.embeddings import get_embedding


# Import OpenAI key
env = environ.Env()
environ.Env.read_env()
API_KEY = env("OPENAI_API_KEY")

# OpenAI Client
client_openai = OpenAI(api_key=API_KEY)

# Embedding model
EMB_MODEL = "text-embedding-3-small"

/home/daniele/Desktop/Projects/dailogy_synthetic_data/.venv_dailogy_sin_data/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_16336/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


## Retrieving Embeddings

In [2]:
def load_examples(path: Path) -> list[dict]:
    # Fetch embedding from sql database
    conn = sqlite3.connect(path)
    cursor = conn.cursor()
    cursor.execute('SELECT id, dysfunctional, embedding, functional FROM examples')
    rows = cursor.fetchall()
    conn.close()

    # Move embedding and text into a list
    examples = []
    for row in rows:
        examples.append({
            'id': row[0],
            'dysfunctional': row[1],
            'embedding': np.array(json.loads(row[2])),
            'functional': row[3]
        })

    return examples

In [3]:
path_emb_db = Path("data_synthetic", "embeddings.db")
examples = load_examples(path_emb_db)

In [4]:
examples[:3]

[{'id': 1,
  'dysfunctional': "You always waste money on useless things, no wonder we're drowning in debt.",
  'embedding': array([ 0.03117449,  0.03328631, -0.00667486, ..., -0.01784991,
          0.00138667,  0.00722167]),
  'functional': "It seems like we spend money on things we don't really need, which is why we're struggling with debt."},
 {'id': 2,
  'dysfunctional': "I can't believe I have to remind you again to pay child support, you're such a deadbeat.",
  'embedding': array([ 0.05065854,  0.01244088, -0.04797346, ...,  0.00821188,
          0.03788203,  0.01860538]),
  'functional': "Hey, could you please remember to make the child support payment? It's really important for our child's well-being. Thank you."},
 {'id': 3,
  'dysfunctional': "You're so irresponsible with money, no wonder our relationship failed.",
  'embedding': array([ 0.03806674,  0.01793936, -0.02905588, ..., -0.01352804,
          0.01850401,  0.02804422]),
  'functional': "I noticed that we had different

In [5]:
print(f"Number of vector embeddings: {len(examples)}")
print(f'Length of vector: {len(examples[0]["embedding"])}')

Number of vector embeddings: 240
Length of vector: 1536


In [6]:
print(f'Dysfunctional text:\n    {examples[0]["dysfunctional"]}')
print(f'Functional text:\n    {examples[0]["functional"]}')

Dysfunctional text:
    You always waste money on useless things, no wonder we're drowning in debt.
Functional text:
    It seems like we spend money on things we don't really need, which is why we're struggling with debt.


## Cosine similarity

The **cosine similarity** between two vectors A and B is calculated as:

$$
\text{cosine\_similarity}(A,B) = \frac{A \cdot B}{\lVert A \rVert \lVert B \rVert}
$$

Where:

- $A \cdot B$ is the dot product of vectors $A$ and $B$.
- $\lVert A \rVert$ and $\lVert B \rVert$ are the Euclidean norms of vectors $A$ and $B$.

In [7]:
def cos_similarity(vec1: np.ndarray, vec2: np.ndarray) -> np.float64:
    """Compute the cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [8]:
input_text_1 = "These are not the droids you are looking for"
input_text_2 = "This is an example to test the function"
input_text_3 = "This sentence is used as example to test the function"
input_embedding_1 = get_embedding(text=input_text_1, model=EMB_MODEL, client=client_openai)
input_embedding_2 = get_embedding(text=input_text_2, model=EMB_MODEL, client=client_openai)
input_embedding_3 = get_embedding(text=input_text_3, model=EMB_MODEL, client=client_openai)

print(f"1 vs. 2 = {cos_similarity(input_embedding_1, input_embedding_2)}")
print(f"1 vs. 3 = {cos_similarity(input_embedding_1, input_embedding_3)}")
print(f"2 vs. 3 = {cos_similarity(input_embedding_2, input_embedding_3)}")

1 vs. 2 = 0.17418072259387657
1 vs. 3 = 0.13943348946249465
2 vs. 3 = 0.8166097013203255


Let's compare the performance of my function with sk-learn's implementation

In [9]:
# Generate random vectors for testing
vec1 = np.random.rand(1000)
vec2 = np.random.rand(1000)

# Benchmark custom function
start_time = time.time()
for _ in range(1000):
    custom_similarity = cos_similarity(vec1, vec2)
custom_time = time.time() - start_time

# Benchmark scikit-learn function
start_time = time.time()
for _ in range(1000):
    sklearn_similarity = cosine_similarity([vec1], [vec2])[0, 0]
sklearn_time = time.time() - start_time

print(f"Custom function time:       {custom_time:.6f} seconds")
print(f"scikit-learn function time: {sklearn_time:.6f} seconds")

# Print the results
print(f"Similarity (custom):       {custom_similarity}")
print(f"Similarity (scikit-learn): {sklearn_similarity}")

Custom function time:       0.014814 seconds
scikit-learn function time: 0.128598 seconds
Similarity (custom):       0.7549164262195733
Similarity (scikit-learn): 0.7549164262195733


## Select closest examples

In [10]:
def find_closest(input_embedding:list, examples:list[dict], top_n:int=5) -> list[dict]:
    """
    Return top_n pairs of dysfunctional text and its functional version,
    based on the cosine similarity with the input_embedding, which is the
    embedding of the text from the user.

     Args:
        input_embedding: Embedding of the user's text.
        examples: List with dictionaries containing dysfunctional text, the dysfunctinal text embedding, and the functional version.
            It has this structure:
                [
                    {'id': 1,
                    'dysfunctional': "A dysfucntional example",
                    'embedding': array([ 0.03117449,  0.03328631, -0.00667486, ..., -0.01784991,]),
                    'functional': "The functional version of the text"},
                    {'id': 2,
                    'dysfunctional': "A dysfucntional example",
                    'embedding': array([ 0.05065854,  0.01244088, -0.04797346, ...,  0.00821188,]),
                    'functional': "The functional version of the text"},
                    ...
                ]
        top_n: number examples to select.

    Returns:
        selected_examples: A list with dictioraries wiht dysfuntional and functional examples.
            It has this structure:
                [
                    {'dysfunctional': "A dysfucntional example",
                    'functional': "The functional version of the text"},
                    {'dysfunctional': "A dysfucntional example",
                    'functional': "The functional version of the text"},
                    ...
                ]
        selected_similarities: A list with the cosine similarities of the selected dysfunctional examples.
            It is calculated as the cosine similarity between the embeddings of user input and those of the
            dysfunctional examples .
            It has this structure:
                [
                    np.float64(0.9999999999999996),
                    np.float64(0.5864792914286562),
                    ...
                ]
    """

    example_embeddings = [example['embedding'] for example in examples]
    similarities = cosine_similarity([input_embedding], example_embeddings)[0]
    similar_indices = similarities.argsort()[-top_n:][::-1]

    selected_examples = [{"dysfunctional":examples[i]["dysfunctional"], "functional":examples[i]["functional"]} for i in similar_indices]

    selected_similarities = [similarities[i] for i in similar_indices]

    return selected_examples, selected_similarities


In [11]:
input_text_1 = "You always waste money on useless things, no wonder we're drowning in debt."
input_embedding_1 = get_embedding(text=input_text_1, model=EMB_MODEL, client=client_openai)

In [12]:
a1, a2 = find_closest(input_embedding_1, examples, 5)

In [13]:
a1

[{'dysfunctional': "You always waste money on useless things, no wonder we're drowning in debt.",
  'functional': "It seems like we spend money on things we don't really need, which is why we're struggling with debt."},
 {'dysfunctional': "Why can't you ever be responsible with our money? You're always spending on nonsense and leaving me to clean up your mess!",
  'functional': "How about we sit down and create a budget together? I feel like we could work on managing our finances more effectively if we both have a say in how we spend our money. Let's find a way to tackle this as a team and avoid any unnecessary stress."},
 {'dysfunctional': "You're so irresponsible with money, no wonder our relationship failed.",
  'functional': "I noticed that we had different approaches to managing finances, which caused some challenges in our relationship. Let's work together to find a better way to handle money in the future."},
 {'dysfunctional': "If you don't want to argue about money, maybe you 

In [14]:
a2

[np.float64(0.9999999999999996),
 np.float64(0.5864792914286562),
 np.float64(0.5382555292872772),
 np.float64(0.49326072504198704),
 np.float64(0.46248037173693135)]

In [15]:
def select_examples(input_text:str, path_emb:Path , emb_model:str, client, num_examples:int=5) -> tuple[list, list]:
    """
    Select the most relevant few-shot examples based on cosine similarity.

    Args:
        data: Dataset with all the text to use to generated the vector embedding.
        path_emb: Path to the .db file with the examples and their embeddings.
        emb_model: Name of the model for the embeddings.
        client: A client for the OpenAI API.
        num_examples: number examples to select.


    Returns:
        dys_text: A list with the dysfuntional examples.
        fun_text: A list with the funtional examples.
    """

    # Embed the user text
    input_embedding = get_embedding(
        text=input_text,
        model=emb_model,
        client=client)
    
    # Load the examples
    examples = load_examples(path_emb)

    # Find the semantically closest example to the input text
    selected_examples, _ = find_closest(input_embedding, examples, num_examples)
   
    return selected_examples

In [16]:
input_text_2 = "You always waste money on things we don't need, no wonder we're drowning in debt."
path_emb_db = Path("data_synthetic", "embeddings.db")

selected_examples = select_examples(
    input_text=input_text_2,
    path_emb=path_emb_db,
    emb_model=EMB_MODEL,
    client=client_openai,
    num_examples=5)

In [17]:
selected_examples

[{'dysfunctional': "You always waste money on useless things, no wonder we're drowning in debt.",
  'functional': "It seems like we spend money on things we don't really need, which is why we're struggling with debt."},
 {'dysfunctional': "Why can't you ever be responsible with our money? You're always spending on nonsense and leaving me to clean up your mess!",
  'functional': "How about we sit down and create a budget together? I feel like we could work on managing our finances more effectively if we both have a say in how we spend our money. Let's find a way to tackle this as a team and avoid any unnecessary stress."},
 {'dysfunctional': "You're so irresponsible with money, no wonder our relationship failed.",
  'functional': "I noticed that we had different approaches to managing finances, which caused some challenges in our relationship. Let's work together to find a better way to handle money in the future."},
 {'dysfunctional': "If you don't want to argue about money, maybe you 

## Dynamic prompt

In [18]:
def create_dynamic_prompt(user_text: str, path_emb:Path , emb_model:str, client, num_examples:int=5) -> str:
    """
    Return a prompt based on the user's text and the selected  examples to enter in the prompt as few-shots.

     Args:
        user_text: The user's text.
        path_emb: Path to the .db file with the examples and their embeddings.
        emb_model: Name of the model for the embeddings.
        client: A client for the OpenAI API.
        num_examples: number examples to select.
        

    Returns:
        A string for the dynamic few-shots prompting.
    """

    # Select the examples with higher cosine similarity with the user text.
    # The cosine similarity is calculated between the embedding of the user's text 
    # and the embeddings of the dysfunctional examples.
    # The selected_examples is a list with dictioraries wiht dysfuntional and functional examples,
    # and has this structure:
    #   [
    #       {'dysfunctional': "A dysfucntional example",
    #       'functional': "The functional version of the text"},
    #       {'dysfunctional': "A dysfucntional example",
    #       'functional': "The functional version of the text"},
    #       ...
    #   ]
    selected_examples = select_examples(
        input_text=user_text,
        path_emb=path_emb,
        emb_model=emb_model,
        client=client,
        num_examples=num_examples)

    prompt_1 = """
    Below is an instruction that describes a task.
    Write a response that appropriately completes the request.
    
    ### Objective:
    Transform the following text, which originates from the context of dysfunctional communication between couples, into functional language.
    Make the text actionable or practical, while maintaining a natural, conversational tone.
    
    ### Instructions:
    1. Review the provided text carefully.
    2. Convert the text into functional, everyday language, focusing on making the content actionable and practical.
    3. Aim for a conversational tone, as if explaining to a friend, to ensure the paragraph is engaging and accessible.
    4. Ensure the transformed text promotes understanding, empathy, and positive communication, suitable for couples or ex-couples who need to interact constructively.
    5. Always respond only with the transformed text and nothing else.
    
    ### Examples 
    Here are some examples of how to convert a dysfucntional text into functional version:
    """

    prompt_2 = ""

    for example in selected_examples:
        example_text = f"""
        - Input: {example["dysfunctional"]}
        - Expected Output: {example["functional"]}
        """
        prompt_2 += example_text


    prompt_3 = f"""
    ### Input 
    Please transform the following text into functional language:
    
    {user_text}
    """

    prompt = prompt_1 + prompt_2 + prompt_3

    return prompt
    

In [19]:
input_text_3 = "You always waste money on things we don't need, no wonder we're drowning in debt."

dyn_prompt = create_dynamic_prompt(
    user_text=input_text_3,
    path_emb=path_emb_db,
    emb_model=EMB_MODEL,
    client=client_openai,
    num_examples=5)

In [20]:
print(dyn_prompt)


    Below is an instruction that describes a task.
    Write a response that appropriately completes the request.
    
    ### Objective:
    Transform the following text, which originates from the context of dysfunctional communication between couples, into functional language.
    Make the text actionable or practical, while maintaining a natural, conversational tone.
    
    ### Instructions:
    1. Review the provided text carefully.
    2. Convert the text into functional, everyday language, focusing on making the content actionable and practical.
    3. Aim for a conversational tone, as if explaining to a friend, to ensure the paragraph is engaging and accessible.
    4. Ensure the transformed text promotes understanding, empathy, and positive communication, suitable for couples or ex-couples who need to interact constructively.
    5. Always respond only with the transformed text and nothing else.
    
    ### Examples 
    Here are some examples of how to convert a dysfucntio